# Train some models from scratch and evaluate them

In [1]:
%load_ext autoreload
%load_ext line_profiler
%load_ext cython
%autoreload 2

import sys, logging
sys.path.append('/notebook')

import embegym
from embegym.data.base import *
from embegym.data.wiki import *

embegym.utils.logger.setupLogger()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/notebook/embegym/evaluation/similarities.py:40: FutureWarning: Conversion of the second argument of issubdtype from `(<class 'float'>, <class 'int'>)` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype((<class 'float'>, <class 'int'>)).type`.
  if numpy.issubdtype(self._gold_data[col].dtype, (float, int))]
2018-02-28 21:46:28,667 - INFO - logger set up


<RootLogger root (DEBUG)>

## Download and preprocess English Wikipedia

In [2]:
# embegym.utils.io.download_if_not_exists('https://dumps.wikimedia.org/enwiki/latest/enwiki-latest-pages-articles.xml.bz2',
#                                         './data/enwiki-latest-pages-articles.xml.bz2')
# cache_corpus_to_pickle(Wikipedia('./data/enwiki-latest-pages-articles.xml.bz2'),
#                        './data/enwiki_full_docs.pickle')

In [3]:
enwiki_full_docs = PickleCorpus('./data/enwiki_full_docs.pickle', verbose=1)
# enwiki_full_docs = LimitedCorpus(enwiki_full_docs, 100)
# TEST_TXT = '''
# мама мыть рама
# рама стоять окно
# папа мыть машина
# окно стоять дом
# гараж стоять дом
# мама жить дом
# папа жить дом
# мама жить папа
# '''
# enwiki_full_docs = [s.strip().split(' ') for s in TEST_TXT.strip().split('\n') if s.strip()]

In [4]:
enwiki_full_docs_sliding_window = SlidingWindow(enwiki_full_docs, left_ctx_size=3, right_ctx_size=3, verbose=0)

HBox(children=(IntProgress(value=0, max=5063314), HTML(value='')))

## Train Word2Vec using Gensim

In [5]:
# gensim_w2v = embegym.models.gensim_wrappers.GensimWord2Vec()
# gensim_w2v.train(enwiki_full_docs)
# gensim_w2v.save('./models/enwiki_w2v_our.gensim')

In [6]:
# gensim_w2v.get_most_similar(gensim_w2v.get_word_vector('mother'))

In [22]:
gensim_w2v = embegym.models.gensim_wrappers.GensimWord2Vec.load('./models/enwiki_w2v_our.gensim')

2018-03-01 09:45:32,910 - INFO - loading Word2Vec object from ./models/enwiki_w2v_our.gensim
2018-03-01 09:45:33,596 - INFO - loading wv recursively from ./models/enwiki_w2v_our.gensim.wv.* with mmap=None
2018-03-01 09:45:33,597 - INFO - loading vectors from ./models/enwiki_w2v_our.gensim.wv.vectors.npy with mmap=None
2018-03-01 09:45:33,642 - INFO - setting ignored attribute vectors_norm to None
2018-03-01 09:45:33,643 - INFO - loading vocabulary recursively from ./models/enwiki_w2v_our.gensim.vocabulary.* with mmap=None
2018-03-01 09:45:33,644 - INFO - loading trainables recursively from ./models/enwiki_w2v_our.gensim.trainables.* with mmap=None
2018-03-01 09:45:33,645 - INFO - loading syn1neg from ./models/enwiki_w2v_our.gensim.trainables.syn1neg.npy with mmap=None
2018-03-01 09:45:33,687 - INFO - setting ignored attribute cum_table to None
2018-03-01 09:45:33,688 - INFO - loaded ./models/enwiki_w2v_our.gensim


## Train Word2Vec using PyTorch

In [7]:
# vocab = embegym.models.vocab.Vocabulary(max_words_final=500000,
#                                         min_count=5)
# embegym.models.vocab.build_vocab_from_dicts_default(vocab, enwiki_full_docs_sliding_window)
# embegym.utils.io.default_save(vocab, './models/enwiki_vocab_500k_5.p')

In [8]:
vocab = embegym.utils.io.default_load('./models/enwiki_vocab_500k_5.p')

In [9]:
def train_my_sgns():
    pt_w2v_sg_ns = embegym.models.pytorch.word2vec.Word2VecSgNs(vector_size=100,
                                                                vocabulary=vocab,
                                                                cuda=True,
                                                                negative=5,
                                                                vocab_min_count=5,
                                                                verbose=1,
                                                                epochs=1)
    pt_w2v_sg_ns.train(enwiki_full_docs_sliding_window)
    pt_w2v_sg_ns.save('./models/enwiki_pt_w2v_sg_ns_our')

In [11]:
# %%prun -s cumulative
train_my_sgns()

2018-02-28 21:46:30,463 - INFO - Init model


HBox(children=(IntProgress(value=0, max=5063314), HTML(value='')))

2018-03-01 09:40:14,103 - INFO - Training interrupted


In [18]:
pt_w2v_sg_ns.get_most_similar(pt_w2v_sg_ns.get_word_vector('kill'))

[('kill', 1.0),
 ('commandos', 0.5561842),
 ('apron', 0.53814065),
 ('police', 0.5342139),
 ('supernatural', 0.5275886),
 ('taser', 0.5275283),
 ('meanwhile', 0.52602595),
 ('guatemalan', 0.5211158),
 ('murder', 0.5175985),
 ('fanny', 0.51558536)]

In [13]:
pt_w2v_sg_ns = embegym.models.pytorch.word2vec.Word2VecSgNs.load('./models/enwiki_pt_w2v_sg_ns_our')

## Evaluate the newly trained models

In [19]:
evaluation = embegym.evaluation.make_evaluation('SimLex999', 'GoogleAnalogies')

In [23]:
eval_result = evaluation(gensim_w2v=gensim_w2v,
                         pt_w2v_sg_ns=pt_w2v_sg_ns)

In [24]:
eval_result.transpose().sort_index()

gensim_w2v  pt_w2v_sg_ns
GoogleAnalogies capital-common-countries @1            NaN      0.005929
                capital-common-countries @5       0.003953      0.023715
                capital-world @1                       NaN      0.000442
                capital-world @5                  0.002653      0.006410
                city-in-state @5                  0.017025           NaN
                family @1                         0.002381           NaN
                family @5                         0.019048      0.002381
                gram1-adjective-to-adverb @1      0.002151           NaN
                gram1-adjective-to-adverb @5      0.013978           NaN
                gram2-opposite @1                 0.001232           NaN
                gram2-opposite @5                 0.004926           NaN
                gram3-comparative @5              0.003754           NaN
                gram5-present-participle @1       0.011089           NaN
                gram5-present-participle @5       0.155242           NaN
                gram6-nationality-adjective @5         NaN      0.003127
                gram7-past-tense @1               0.003205           NaN
                gram7-past-tense @5               0.033333      0.002564
                gram8-plural @1                   0.010511      0.000751
                gram8-plural @5                   0.093093      0.004505
                micro-mean @1                     0.001854      0.000326
                micro-mean @5                     0.022685      0.003097
                morphological @1                  0.007724      0.000257
                morphological @5                  0.084964      0.002575
                semantic @1                       0.000115      0.000573
                semantic @5                       0.007335      0.004814
                syntactic @1                      0.000524      0.000000
                syntactic @5                      0.003840      0.000863
SimLex999       Assoc(USF)                        0.303414      0.121656
                SD(SimLex)                        0.004529      0.004153
                SimLex999                         0.290432      0.108370
                conc(w1)                          0.003866      0.032516
                conc(w2)                         -0.001712      0.033601